In [2]:
import sys

In [3]:
print(sys.executable)

/Users/rliao/opt/anaconda3/bin/python


## Download data from BigQuery

In [1]:
from google.cloud import bigquery
from google.cloud import storage
from google_auth_oauthlib import flow
import pandas as pd
#import pandas_gbq
#from datalab.context import Context

In [2]:
# RLiao account json credentials
client_secrets_json = '/Users/rliao/Credentials/client_secret_394671187685-qjjdbmab2a7m413p1rgq8fuv8lv0iufi.apps.googleusercontent.com.json'

# project
project_id = 'scg-dai-sci-dev'
dataset_id = 'Playground_rliao'
# scopes
scopes = ['https://www.googleapis.com/auth/bigquery',
          'https://www.googleapis.com/auth/cloud-platform']

In [3]:
def authenticate_credentials(client_secrets_json, scopes):
    """Prepare credentials for authentication
    :rtype: credentials object
    """
    print('...authenticating user')
    appflow = flow.InstalledAppFlow.from_client_secrets_file(client_secrets_json, scopes)
    appflow.run_local_server()
    credentials = appflow.credentials
    # return
    return(credentials)

# authenticate user and get credentials
credentials = authenticate_credentials(client_secrets_json= client_secrets_json,
                                       scopes = scopes)

...authenticating user
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=394671187685-qjjdbmab2a7m413p1rgq8fuv8lv0iufi.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=5pRjDUvRBQOC4S9u0oy2tDfPbwSA4J&access_type=offline


In [6]:
#client = bigquery.Client.from_service_account_json(service_account_json)
bqclient = bigquery.Client(project= project_id, credentials= credentials)
#bqstorageclient = bigquery_storage_v1beta1.BigQueryStorageClient(credentials=credentials)

In [4]:
# Reading query
query_string = """

Select *from visit_cluster
"""

#print('...preparing query')
#fd = open('../sql/get_turnover_data_for_audit.sql','r')
#sqlFile = fd.read()

In [7]:
df = bqclient.query(query_string).to_dataframe()

#dataframe = (
 #   bqclient.query(query_string)
  #  .result()
   # .to_dataframe(bqstorage_client=bqstorageclient)
#)
# print(dataframe.head())

## Upload data to BigQuery

In [50]:
nsw_covid19_url = 'https://data.nsw.gov.au/data/dataset/aefcde60-3b0c-4bc0-9af1-6fe652944ec2/resource/21304414-1ff1-4243-a5d2-f52778048b29/download/covid-19-cases-by-notification-date-and-postcode-local-health-district-and-local-government-area.csv'
# download data
df_nsw = pd.read_csv(nsw_covid19_url)

In [4]:
df_nsw.head()

,notification_date,postcode,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19
0,2020-01-22,2134.0,X700,Sydney,11300.0,Burwood (A)
1,2020-01-24,2121.0,X760,Northern Sydney,16260.0,Parramatta (C)
2,2020-01-25,2033.0,X720,South Eastern Sydney,16550.0,Randwick (C)
3,2020-01-25,2071.0,X760,Northern Sydney,14500.0,Ku-ring-gai (A)
4,2020-02-21,2010.0,X720,South Eastern Sydney,17200.0,Sydney (C)


In [44]:
def upload_to_bq(dataset_id, table_name, df): #takes dataset_id, how you want to name it in bq, df as argument
    df.to_gbq(dataset_id+'.'+table_name, 
                project_id, # Context.default().project_id,
                chunksize=1000, 
                if_exists='replace' #append,fail
            )

In [51]:
upload_to_bq(dataset_id='Playground_rliao', table_name='nsw_covid_19_lga', df=df_nsw)

#Pranam's old approach
#table_ref = client.dataset(dataset_id).table('nsw_covid19_data_tmp')
#client.load_table_from_dataframe(df, table_ref).result()

3it [00:21,  7.17s/it]


In [39]:
json_list = ['https://interactive.guim.co.uk/covidfeeds/nsw.json',
             'https://interactive.guim.co.uk/covidfeeds/victoria.json',
             'https://interactive.guim.co.uk/covidfeeds/queensland.json',
             'https://interactive.guim.co.uk/covidfeeds/wa.json']

name_list = ['nsw_covid_feeds', 'vic_covid_feeds','qld_covid_feeds','wa_covid_feeds']

#Create a df that holds all the json links and how you would name them
df_json = pd.DataFrame(list(zip(name_list, json_list)), columns =['tableName', 'link']) 

In [40]:
def json_to_df(df): #pass through a df that holds all json
    
    df_collection = {} #initiate empty dictionary to hold all resulted df
    
    for i in range(len(df.iloc[:,1])):
        df_name= df['tableName'][i]
        df_collection[df_name]=pd.read_json(df['link'][i], orient='records')
        
    return df_collection

In [41]:
#call function to create the dict that holds all df
df_collection=json_to_df(df=df_json)

In [49]:
#looping through each df and upload to bq
for i in df_collection:
    upload_to_bq(dataset_id='Playground_rliao', table_name=i, df=df_collection[i])

1it [00:06,  6.76s/it]
1it [00:06,  6.54s/it]
1it [00:06,  6.24s/it]
1it [00:06,  6.48s/it]
